In [26]:
import config # this is the api file
import pandas as pd
import numpy as np
import finnhub
import time
import datetime
from operator import itemgetter

In [23]:
#start and end date for the data
dt_start = "01/04/2022"
dt_end = "11/04/2022"

#convert the date data into unix timestamp
utc_start = int(datetime.datetime.strptime(dt_start, "%d/%m/%Y").timestamp())
utc_end = int(datetime.datetime.strptime(dt_end, "%d/%m/%Y").timestamp())

In [24]:
# list of stocks
stocks = ['AAPL','MSFT','GOOG','AMZN']

In [33]:
finnhub_client = finnhub.Client(api_key=config.finnhub_key)

def temp(stocks):
    for stock in stocks:
        print(finnhub_client.stock_candles(stock, 'D', utc_start, utc_end))

In [34]:
print(temp(stocks))

{'c': [174.31, 178.44, 175.06, 171.83, 172.14, 170.09, 165.75], 'h': [174.88, 178.49, 178.3, 173.63, 173.36, 171.78, 169.03], 'l': [171.94, 174.44, 174.415, 170.13, 169.85, 169.2, 165.5], 'o': [174.03, 174.57, 177.5, 172.36, 171.16, 171.78, 168.71], 's': 'ok', 't': [1648771200, 1649030400, 1649116800, 1649203200, 1649289600, 1649376000, 1649635200], 'v': [78751328, 76545983, 73401786, 89058782, 77594650, 76575508, 72246706]}
{'c': [309.42, 314.97, 310.88, 299.5, 301.37, 296.97, 285.26], 'h': [310.13, 315.11, 314.865, 307, 303.65, 301.12, 292.61], 'l': [305.54, 309.71, 309.87, 296.71, 296.35, 296.28, 285], 'o': [309.37, 310.09, 313.27, 305.19, 296.66, 300.44, 291.79], 's': 'ok', 't': [1648771200, 1649030400, 1649116800, 1649203200, 1649289600, 1649376000, 1649635200], 'v': [27110529, 24335868, 23156719, 40110372, 31411155, 24361917, 34569264]}
{'c': [2814, 2872.85, 2821.26, 2743.52, 2729.3, 2680.21, 2595.93], 'h': [2819, 2880.875, 2871.8, 2796.97, 2754.03, 2725, 2658.7836], 'l': [2775.9